### Initial Setup

In [1]:
from lsmgridtrack.test import data
import lsmgridtrack as lsm

# Get paths to reference and deformed images
reference_path = data.get_image('reference - 2 layers')
deformed_path = data.get_image('10 percent strain - 2 layers')

# Instantiate a tracker object with image paths and configuration file specified as arguments
t = lsm.tracker(reference_path = reference_path,
                deformed_path = deformed_path,
                config = "example2.yaml")

# Print the options to show they are changed by the config file
print(t.options)


{'Image': {'spacing': [0.5, 0.5, 1.0], 'resampling': [1.0, 1.0, 1.0]}, 'Grid': {'origin': [69, 72, 5], 'spacing': [20, 20, 10], 'size': [20, 20, 3], 'crop': False}, 'Registration': {'method': 'BFGS', 'iterations': 100, 'sampling_fraction': 0.05, 'sampling_strategy': 'RANDOM', 'usemask': False, 'landmarks': [[72, 81, 5], [71, 467, 5], [457, 468, 5], [455, 82, 5], [71, 80, 20], [72, 468, 20], [458, 466, 20], [457, 80, 20]], 'shrink_levels': [1], 'sigma_levels': [0.0]}}


### Execution

In [2]:
t.execute()

... Starting Deformable Registration
... ... Finding optimal BSpline transform
... ... Elapsed Iterations: 0
... ... Current Metric Value: -2.34740E-01
... ... Elapsed Iterations: 0
... ... Current Metric Value: -2.65271E-01
... ... Elapsed Iterations: 0
... ... Current Metric Value: -3.87429E-01
... ... Elapsed Iterations: 0
... ... Current Metric Value: -3.87429E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -1.61627E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -4.17114E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -5.04918E-01
... ... Elapsed Iterations: 1
... ... Current Metric Value: -5.04918E-01
... ... Elapsed Iterations: 2
... ... Current Metric Value: -5.66540E-01
... ... Elapsed Iterations: 2
... ... Current Metric Value: -5.66540E-01
... ... Elapsed Iterations: 3
... ... Current Metric Value: -5.77310E-01
... ... Elapsed Iterations: 3
... ... Current Metric Value: -5.77310E-01
... ... Elapsed Iterations: 4
... ... Current

### Outputting Results

In [3]:
t.writeResultsAsVTK('example2')
t.writeResultsAsExcel('example2')
t.writeResultsAsNumpy('example2')

... Saving Results to example2.vti
... Saving Results to example2.xlsx
... Saving file as numpy archive example2.npz


### Post-analysis

In [4]:
data = lsm.utils.readVTK("example2.vti")

Here *data* is an OrderedDict that works with the post-processing functions included in lsm.utils. All the reader functions return this type, so we could also have read the excel file we output above:

In [5]:
excel_data = lsm.utils.readExcel("example2.xlsx")

Now if we have deformation data from images with grids of the same size, we can do one-to-one comparisons. Since in this example we only have results from one analysis, we will simulate a second data set by randomly perturbing a copy of our *data*. Note that this deepcopy is important, so nothing is copied by reference.

In [6]:
import numpy as np
from copy import deepcopy
data_perturbed = deepcopy(data)

Let's add some random pertubation to the values stored in our copy of the data.

In [7]:
for k, v in data_perturbed.items():
    if k == "Coordinates":
        continue
    mu = np.mean(v.ravel())
    sigma = np.std(v.ravel())
    data_perturbed[k] += np.random.normal(loc=mu, scale=sigma, size=v.shape)

A nice aggregate measure is the root-mean-square difference. We can easily calculate this for all data variables.

In [8]:
rmsd = lsm.utils.calculateRMSDifference(x=data, y=data_perturbed, variables=data.keys())
print(rmsd)

OrderedDict([('Coordinates', 0.0), ('Displacement', 4.3950385988639971), ('Strain', 0.099083840270760157), ('1st Principal Strain', 0.013016456504172057), ('2nd Principal Strain', 0.0070212611135075983), ('3rd Principal Strain', 0.13202721704346612), ('Volumetric Strain', 0.18459445802160926), ('Maximum Shear Strain', 0.1692308333647991)])


However, all spatial information is lost here. Instead we can calculate the difference at every grid vertex, and save it to a variable. 

In [9]:
strain_difference = lsm.utils.calculateDifference(x=data, y=data_perturbed, variable="Strain")

In [10]:
data["Strain Difference"] = strain_difference

Of course this could be done in one line.

In [11]:
data["1st Principal Strain Difference"] = lsm.utils.calculateDifference(x=data, y=data_perturbed, variable="1st Principal Strain")

Now, it would be nice to visualize this again in ParaView (or other software that can handle VTK image format). We supply writer functions to VTK, NumPy, and Excel formats as well. Let's write to a VTK image.

In [12]:
lsm.utils.writeAsVTK(data=data, name="example2_processed")

... Wrote grid data to example2_processed.vti
